<a href="https://colab.research.google.com/github/mireia-j/genetic-algorithm-deliveries/blob/main/supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handout 6 - Machine Learning
**Authors:** Mireia Jaume and Carles Westendorf

The first step is to import the data and get a glimpse of what kind of data we have.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/ds_05.csv')
print('**** DF ORIGINAL ****')
print(df.info())

**** DF ORIGINAL ****
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9811 entries, 0 to 9810
Columns: 160 entries, Unnamed: 0 to class
dtypes: float64(127), int64(29), object(4)
memory usage: 12.0+ MB
None


We have 160 columns and 9811 samples. There are 156 columns that are numeric and 4 columns that are another type of data.

## Data cleaning

The first thing we are going to do is to check for duplicates.

The first thing we will do is to **delete the columns that we do not need**. These columns are the ones related to identification and timestamping data.

In [ ]:
df1 = df.copy(deep=True)

duplicated_rows = df1[df1.duplicated()]

if not duplicated_rows.empty:
  print(f"The duplicated rows are {duplicated_rows}")
else:
  print("There are no duplicates in our dataframe")

There are no duplicates in our dataframe


In [ ]:
cols_to_drop = ["Unnamed: 0", "user_name", "raw_timestamp_part_1", "raw_timestamp_part_2", "cvtd_timestamp", "new_window", "num_window"]

# errors='ignore' prevents an error if we accidentally run this cell twice
df1 = df1.drop(columns=cols_to_drop, errors='ignore')

print(f"We are left with {df1.shape[1]} columns")

We are left with 153 columns


We now want to check which columns have a **high percentage of NA values and delete those columns**. The criteria is that if a column has more that 90% of NA values, we delete that column from the original dataframe. We can do so because they do not provide enough data.

In [ ]:
missing_percentages = df1.isna().mean()

high_na_columns = missing_percentages[missing_percentages > 0.9]

df1 = df1.drop(columns=high_na_columns.index)

print(f"We have {len(high_na_columns)} columns that have more than 90% of NA values")
print(f"After deleting the columns, the dataframe has now {df1.shape[1]} columns")

We have 100 columns that have more than 90% of NA values
After deleting the columns, the dataframe has now 53 columns


The next step is to check which columns still have NA values and deal with the missing values.

In [ ]:
missing_percentages = df1.isna().mean()

some_na_columns = missing_percentages[missing_percentages > 0]

print(f"We have {len(some_na_columns)} columns that still have NA values")

We have 0 columns that still have NA values


We do not have to transform the data. Therefore, let us focus now on the **outliers**. We will use the inter-quartile range method for non-Gausssian data to detect outliers.

Some interesting questions that we could be asking ourselves are:
1.   Is there any feature that has many outliers? Maybe we could consider deleting that column.
2.   Is there an specific sample that has as outliers many columns? Maybe we could consider removing that sample from the data.
3.   Is there any class that has more outliers than the others? Watch out for that class.



In [ ]:
feature_cols = df1.columns[:-1]
label_col = df1.columns[-1]

Q1 = df1[feature_cols].quantile(0.25)
Q3 = df1[feature_cols].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# We want to create a dataframe containing the sample that has an outlier
# To which class that outlier belongs and the columns where it has the outlier
outlier_mask = (df1[feature_cols] < lower_bound) | (df1[feature_cols] > upper_bound)

rows_with_outliers = outlier_mask.any(axis=1)
subset_df = df1[rows_with_outliers]

results = []
for index, row in subset_df.iterrows():
    row_mask = outlier_mask.loc[index]
    outliers_dict = row[feature_cols][row_mask].to_dict()

    results.append({
        'sample': index,
        'type_class': row[label_col],
        'columns_outliers': outliers_dict
    })

df_outliers = pd.DataFrame(results)

# We want to create a dictionary that counts and sorts the outliers per column
counts_series = outlier_mask.sum()

counts_series = counts_series[counts_series > 0]

sorted_counts = counts_series.sort_values(ascending=False)

outlier_counts_sorted = sorted_counts.to_dict()

print("Sorted number of outliers per column (column name: count):")
print(outlier_counts_sorted)

print("\nDetailed DataFrame Preview:")
df_outliers[:2]

Sorted number of outliers per column (column name: count):
{'accel_belt_x': 2192, 'pitch_belt': 2034, 'magnet_belt_x': 2000, 'magnet_dumbbell_x': 1456, 'magnet_dumbbell_y': 1368, 'gyros_belt_x': 1360, 'gyros_dumbbell_y': 1359, 'accel_dumbbell_x': 975, 'pitch_forearm': 776, 'magnet_dumbbell_z': 766, 'gyros_belt_z': 737, 'gyros_dumbbell_x': 614, 'magnet_belt_y': 579, 'yaw_belt': 571, 'gyros_dumbbell_z': 466, 'gyros_arm_y': 442, 'magnet_forearm_z': 433, 'total_accel_forearm': 430, 'magnet_belt_z': 396, 'magnet_arm_z': 367, 'accel_arm_z': 313, 'gyros_forearm_z': 286, 'pitch_arm': 240, 'gyros_belt_y': 185, 'accel_forearm_y': 162, 'gyros_forearm_x': 150, 'accel_dumbbell_z': 63, 'gyros_arm_z': 61, 'total_accel_arm': 28, 'pitch_dumbbell': 24, 'gyros_forearm_y': 17, 'gyros_arm_x': 9, 'roll_dumbbell': 7, 'accel_dumbbell_y': 5, 'accel_belt_y': 3, 'yaw_arm': 3, 'total_accel_dumbbell': 2}

Detailed DataFrame Preview:


,sample,type_class,columns_outliers
0,1,A,"{'pitch_belt': -39.8, 'accel_belt_x': 47, 'mag..."
1,2,A,"{'accel_dumbbell_x': -203, 'accel_dumbbell_z':..."


In [ ]:
df_outliers['outlier_count'] = df_outliers['columns_outliers'].apply(len)

top = 11
top_samples = df_outliers.nlargest(top, 'outlier_count')

print(f"Top {top} samples with the most outliers:")
print(top_samples['outlier_count'])

Top 11 samples with the most outliers:
3094    12
2308    10
2370    10
2585    10
2859    10
4074    10
4128    10
5389    10
6013    10
7006    10
88       9
Name: outlier_count, dtype: int64


In [ ]:
class_counts = df_outliers['type_class'].value_counts()

print("Number of samples per class that have outliers:")
print(class_counts)

Number of samples per class that have outliers:
type_class
A    2120
E    1668
B    1538
C    1044
D     889
Name: count, dtype: int64


Here we can see how many unique values we have per column. We have only printed the columns that have less than 200 different values.

In [ ]:
# df1.nunique()[df1.nunique() < 200]

We can now proceed to **normalize** our data.

## Feature engineering